In [98]:
import os
R_PATH = "/home/jdkent/envs/aim1_valid/lib/R"
os.environ['R_HOME'] = R_PATH
import matplotlib.pyplot as plt
import pandas as pd
import nibabel as nib
import numpy as np
from bids.layout import BIDSLayout
from multiprocessing.pool import Pool
import re
import notebook_functions as nf
from nilearn.input_data import NiftiLabelsMasker
from itertools import product

In [125]:
N_THREADS = 32
LSS_PATH = '../lssNoSignalScale/nibetaseries'
LSA_PATH = '../lsaNoSignalScale/nibetaseries'
FMRIPREP_PATH = '../fmriprep'
BIDS_PATH = '../..'
RESPONSE_ATLAS = '../data/overall_response_atlas.nii.gz'
RESPONSE_LUT = '../data/overall_response.tsv'
# path to bold QA measures
BOLD_QA = '../mriqc/group_bold.tsv'
SCHAEFER_ATLAS = '../data/Schaefer2018_400Parcels_17Networks_order_FSLMNI152_2mm.nii.gz'
SCHAEFER_LUT = '../data/schaefer_parcel-400_network-17.tsv'

In [9]:
def proc_bold_qa(bold_qa_file):
    bold_qa = pd.read_csv(bold_qa_file, sep='\t')
    # drop the rest rows
    bold_qa = bold_qa[~bold_qa['bids_name'].str.contains('.*rest.*')]
    
    split_columns = bold_qa['bids_name'].str.split('_|-', n = 7, expand = True)
    bold_qa['task'] = split_columns[5]
    bold_qa['participant_id'] = split_columns[1]
    return bold_qa

bold_qa = proc_bold_qa(BOLD_QA)
bold_qa.head()

,bids_name,aor,aqi,dummy_trs,dvars_nstd,dvars_std,dvars_vstd,efc,fber,fd_mean,...,summary_fg_mad,summary_fg_mean,summary_fg_median,summary_fg_n,summary_fg_p05,summary_fg_p95,summary_fg_stdv,tsnr,task,participant_id
0,sub-GE120001_ses-pre_task-fauxbold_bold,0.001063,0.006351,2,16.187316,1.198663,1.010760,0.448331,3009467.75,0.103818,...,232.520432,1381.902222,1410.488159,29195.0,837.753119,1812.366785,297.530945,64.075073,fauxbold,GE120001
3,sub-GE120001_ses-pre_task-taskswitch_bold,0.003942,0.005572,2,15.734272,1.148948,0.957532,0.450887,1364181.25,0.138379,...,216.930939,1391.725708,1421.432861,29084.0,868.629898,1788.999225,284.924835,72.902779,taskswitch,GE120001
4,sub-GE120002_ses-pre_task-fauxbold_bold,0.002056,0.010868,1,17.975215,1.024274,1.143355,0.432753,3026978.75,0.227028,...,225.343140,1283.572998,1322.169800,27338.0,738.498715,1686.504254,287.157501,47.578674,fauxbold,GE120002
7,sub-GE120002_ses-pre_task-taskswitch_bold,0.002134,0.004827,1,16.918552,1.176876,0.990735,0.431836,4213331.00,0.169619,...,229.552505,1292.104004,1334.634033,27340.0,734.595834,1700.930396,292.718201,71.898445,taskswitch,GE120002
8,sub-GE120003_ses-pre_task-fauxbold_bold,0.003094,0.018135,1,38.223070,1.116687,0.909317,0.393451,2381240.50,0.577373,...,246.342178,1247.227295,1309.490601,23125.0,595.968738,1648.415723,314.057434,31.255548,fauxbold,GE120003


In [62]:
def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False
    otherwise.
    modified from nipype:
    https://github.com/nipy/nipype/blob/b62d80/nipype/algorithms/confounds.py#L1129
    Parameters
    ----------
    points: nparray
        an numobservations by numdimensions numpy array of observations
    thresh: float
        the modified z-score to use as a threshold. Observations with
        a modified z-score (based on the median absolute deviation) greater
        than this value will be classified as outliers.
    Returns
    -------
        A bolean mask, of size numobservations-length array.
    .. note:: References
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor.
    """
    import numpy as np

    if len(points.shape) == 1:
        points = points[:, None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [10]:
bold_qa_select = bold_qa[['participant_id', 'task', 'tsnr', 'fd_mean', 'fd_num']]
# eliminate people with fd_num >= 100 (threshold to keep number of regressors low)
bad_participants = bold_qa_select[bold_qa_select['fd_num'] >= 100]['participant_id'].unique()
bad_participants

array(['GE120003', 'GE120015', 'GE120020', 'GE120021', 'GE120024',
       'GE120026', 'GE120037', 'GE120038', 'GE120039', 'GE120043',
       'GE120045', 'GE120049', 'GE120052', 'GE120054', 'GE120055',
       'GE120059', 'GE120060', 'GE120065', 'GE120068', 'GE120069',
       'GE120075'], dtype=object)

In [111]:
lss_layout = BIDSLayout(LSS_PATH, validate=False, config=['bids', 'derivatives'])
lsa_layout = BIDSLayout(LSA_PATH, validate=False, config=['bids', 'derivatives'])

In [112]:
ent = lss_layout.entities['subject']
good_participants = list(set(ent.unique()) - set(bad_participants))
len(good_participants)

40

In [113]:
lsa_betas = nf.get_layout_objects(lsa_layout, trialtypes=['single', 'repeat', 'switch'], 
                      suffix="betaseries", extension="nii.gz", task="taskswitch",
                      subject=good_participants)
lsa_residuals = lsa_layout.get(task="taskswitch", desc='residuals', subject=good_participants)

lss_betas = nf.get_layout_objects(lss_layout, trialtypes=['single', 'repeat', 'switch'], 
                      suffix="betaseries", extension="nii.gz", task="taskswitch",
                    subject=good_participants)
lss_residuals = lsa_layout.get(task="taskswitch", desc='residuals', subject=good_participants)

In [114]:
response_masker = NiftiLabelsMasker(RESPONSE_ATLAS)
response_lut = pd.read_csv(RESPONSE_LUT, sep='\t')
response_lut

,regions,index
0,1,1
1,1a,2
2,1b,3
3,1c,4
4,2,5
5,2a,6
6,2b,7
7,2c,8
8,3,9
9,3a,10


In [126]:
schaefer_masker = NiftiLabelsMasker(SCHAEFER_ATLAS)
schaefer_lut = pd.read_csv(SCHAEFER_LUT, sep='\t')

In [127]:
schaefer_lut

,regions,index
0,LH-VisCent-ExStr_1,1
1,LH-VisCent-ExStr_2,2
2,LH-VisCent-ExStr_3,3
3,LH-VisCent-ExStr_4,4
4,LH-VisCent-ExStr_5,5
5,LH-VisCent-ExStr_6,6
6,LH-VisCent-ExStr_7,7
7,LH-VisCent-ExStr_8,8
8,LH-VisCent-ExStr_9,9
9,LH-VisCent-ExStr_10,10


In [129]:
def process_participant_betas(layout, lsa_layout, participant, atlas):
    # get the betas for the relavent trial types
    betas = nf.get_layout_objects(
        layout, trialtypes=['single', 'repeat', 'switch'], 
        suffix="betaseries", extension="nii.gz", task="taskswitch",
        subject=participant)
    
    # get the residual from LSA no matter what since the residual comes from one model
    # (as opposed to the average of residuals that LSS returns)
    residual = lsa_layout.get(task="taskswitch", desc='residuals', subject=participant)
    
    # get the average time/beta series from each region of interest
    masker = NiftiLabelsMasker(atlas)
    
    # get the betas for all trial types
    all_betas = [masker.fit_transform(betas[cond][0].path) for cond in ['single', 'repeat', 'switch']]
    
    # filter the beta series from outliers
    filtered_betas = [beta_data[~is_outlier(beta_data)] for beta_data in all_betas]
    
    # get the mean amplitude (across trial types) for each region of interest
    amplitude_betas = np.nanmean(np.array([np.nanmedian(np.abs(beta_data), axis=0) for beta_data in filtered_betas]), axis=0)
    
    # get the mean standard deviation (across trial types) for each region of interest
    std_betas = np.nanmean(np.array([np.nanstd(beta_data, axis=0) for beta_data in filtered_betas]), axis=0)
    
    # get the standard deviation of the noise for each region of interest
    std_noise = np.nanstd(masker.fit_transform(residual[0].path), axis=0)
    
    # contrast to noise ratio
    cnr = amplitude_betas / std_noise
    
    # contrast variance to noise ratio
    cvnr = std_betas / std_noise
    
    return cnr, cvnr
    

In [148]:
cnr_activation_collector = {}
cvnr_activation_collector = {}
for method, ly in [("lsa", lsa_layout), ("lss", lss_layout)]:
    cnr_activation_collector[method] = {}
    cvnr_activation_collector[method] = {}
    for participant in good_participants:
        cnr, cvnr = process_participant_betas(ly, lsa_layout, participant, RESPONSE_ATLAS)
        cnr_activation_collector[method][participant] = cnr
        cvnr_activation_collector[method][participant] = cvnr

In [149]:
cvnr_activation_lsa_df = pd.DataFrame.from_dict(cvnr_activation_collector['lsa'], orient='index', columns=response_lut['regions'])
cnr_activation_lsa_df = pd.DataFrame.from_dict(cnr_activation_collector['lsa'], orient='index', columns=response_lut['regions'])
cvnr_activation_lss_df = pd.DataFrame.from_dict(cvnr_activation_collector['lss'], orient='index', columns=response_lut['regions'])
cnr_activation_lss_df = pd.DataFrame.from_dict(cnr_activation_collector['lss'], orient='index', columns=response_lut['regions'])

In [150]:
cvnr_activation_lsa_df.describe().T

,count,mean,std,min,25%,50%,75%,max
regions,,,,,,,,
1,40.0,1.493791,0.698882,0.603026,0.957895,1.400787,1.642858,3.531632
1a,40.0,1.446815,0.465670,0.713626,1.191430,1.335361,1.575823,3.183634
1b,40.0,1.646220,0.709622,0.531828,1.032060,1.598571,1.994703,3.565423
1c,40.0,1.937784,1.034760,0.560799,1.378113,1.650934,2.272146,6.161914
2,40.0,2.856246,1.711651,0.768488,1.550718,2.468440,3.520120,7.791657
2a,40.0,1.378601,0.835340,0.431093,0.937165,1.223010,1.490604,4.598309
2b,40.0,3.133568,1.589274,1.402476,2.140411,2.601349,3.407890,8.884598
2c,40.0,1.534577,1.166180,0.330930,0.944795,1.233039,1.747221,6.610168
3,40.0,2.540929,1.161353,0.753146,1.526721,2.348683,3.445912,5.595266


In [156]:
cvnr_act_lsa_mean = cvnr_activation_lsa_df.describe().T['mean'].mean()
cvnr_act_lsa_max = cvnr_activation_lsa_df.describe().T['mean'].max()
print("AVNR (activation, lsa) Mean: {}".format(cvnr_act_lsa_mean))
print("AVNR (activation, lsa) Max: {}".format(cvnr_act_lsa_max))

AVNR (activation, lsa) Mean: 2.15341683963574
AVNR (activation, lsa) Max: 3.260426728745448


In [151]:
cnr_activation_lsa_df.describe().T

,count,mean,std,min,25%,50%,75%,max
regions,,,,,,,,
1,40.0,1.013019,0.486004,0.397652,0.702270,0.854607,1.182677,2.830594
1a,40.0,1.010043,0.330937,0.500087,0.788534,0.994951,1.118403,2.079824
1b,40.0,1.103887,0.452523,0.417274,0.780378,1.009616,1.259987,2.647754
1c,40.0,1.332659,0.653980,0.418005,0.903863,1.146819,1.656428,3.754581
2,40.0,1.954379,1.135411,0.592758,1.104842,1.754123,2.306997,5.183483
2a,40.0,0.923844,0.546594,0.321112,0.654297,0.823290,1.006208,3.348192
2b,40.0,2.176973,1.152482,0.936605,1.527462,1.821859,2.433338,6.545389
2c,40.0,0.990283,0.700150,0.225749,0.625231,0.800554,1.119164,3.540455
3,40.0,1.725172,0.738436,0.570616,1.072489,1.550546,2.323591,3.622044


In [157]:
cnr_act_lsa_mean = cnr_activation_lsa_df.describe().T['mean'].mean()
cnr_act_lsa_max = cnr_activation_lsa_df.describe().T['mean'].max()
print("CNR (activation, lsa) Mean: {}".format(cnr_act_lsa_mean))
print("CNR (activation, lsa) Max: {}".format(cnr_act_lsa_max))

CNR (activation, lsa) Mean: 1.4151391926183445
CNR (activation, lsa) Max: 2.1769726814544077


In [152]:
cvnr_activation_lss_df.describe().T

,count,mean,std,min,25%,50%,75%,max
regions,,,,,,,,
1,40.0,0.991699,0.459659,0.402916,0.672556,0.839920,1.197684,2.342959
1a,40.0,1.010297,0.383920,0.535665,0.765383,0.897015,1.115829,2.503862
1b,40.0,1.042829,0.337555,0.300165,0.804933,1.016416,1.264432,1.740425
1c,40.0,1.168353,0.445155,0.348067,0.903559,1.083729,1.431145,2.441960
2,40.0,1.862532,1.030380,0.469624,1.108758,1.619184,2.312283,4.484069
2a,40.0,0.913721,0.513803,0.289238,0.574667,0.796160,1.099487,2.916147
2b,40.0,2.115082,0.981789,1.050287,1.374604,1.792286,2.535464,5.011984
2c,40.0,1.036974,0.804704,0.213250,0.634449,0.833476,1.129677,4.723067
3,40.0,1.713144,0.916288,0.513759,1.101005,1.459309,2.359862,5.240617


In [158]:
cvnr_act_lss_mean = cvnr_activation_lss_df.describe().T['mean'].mean()
cvnr_act_lss_max = cvnr_activation_lss_df.describe().T['mean'].max()
print("AVNR (activation, lss) Mean: {}".format(cvnr_act_lss_mean))
print("AVNR (activation, lss) Max: {}".format(cvnr_act_lss_max))

AVNR (activation, lss) Mean: 1.421244311103253
AVNR (activation, lss) Max: 2.177198372941991


In [153]:
cnr_activation_lss_df.describe().T

,count,mean,std,min,25%,50%,75%,max
regions,,,,,,,,
1,40.0,0.733415,0.328015,0.323872,0.501764,0.608512,0.873120,1.562037
1a,40.0,0.773206,0.260932,0.406581,0.611929,0.700751,0.877462,1.678054
1b,40.0,0.697209,0.254617,0.216813,0.552763,0.689467,0.783594,1.734160
1c,40.0,0.857797,0.375526,0.245431,0.623866,0.778822,0.951853,2.261710
2,40.0,1.364370,0.749001,0.328800,0.768002,1.139188,1.655693,3.222138
2a,40.0,0.742210,0.594321,0.198677,0.455783,0.588144,0.756345,3.666304
2b,40.0,1.581858,0.806183,0.621146,1.021184,1.408843,1.820071,3.944619
2c,40.0,0.703025,0.492417,0.124908,0.412172,0.588418,0.803159,2.284423
3,40.0,1.179387,0.688070,0.382260,0.718691,1.001191,1.530680,4.435945


In [159]:
cnr_act_lss_mean = cnr_activation_lss_df.describe().T['mean'].mean()
cnr_act_lss_max = cnr_activation_lss_df.describe().T['mean'].max()
print("CNR (activation, lss) Mean: {}".format(cnr_act_lss_mean))
print("CNR (activation, lss) Max: {}".format(cnr_act_lss_max))

CNR (activation, lss) Mean: 0.9895698447616426
CNR (activation, lss) Max: 1.5818575998173128


In [162]:
cnr_schaefer_collector = {}
cvnr_schaefer_collector = {}
for method, ly in [("lsa", lsa_layout), ("lss", lss_layout)]:
    cnr_schaefer_collector[method] = {}
    cvnr_schaefer_collector[method] = {}
    for participant in good_participants:
        cnr, cvnr = process_participant_betas(ly, lsa_layout, participant, SCHAEFER_ATLAS)
        cnr_schaefer_collector[method][participant] = cnr
        cvnr_schaefer_collector[method][participant] = cvnr

/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in true_divide
/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in true_divide
/home/jdkent/.conda/envs/aim1_valid/lib/python3.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid valu

In [164]:
cvnr_schaefer_lsa_df = pd.DataFrame.from_dict(cvnr_schaefer_collector['lsa'], orient='index', columns=schaefer_lut['regions'])
cnr_schaefer_lsa_df = pd.DataFrame.from_dict(cnr_schaefer_collector['lsa'], orient='index', columns=schaefer_lut['regions'])
cvnr_schaefer_lss_df = pd.DataFrame.from_dict(cvnr_schaefer_collector['lss'], orient='index', columns=schaefer_lut['regions'])
cnr_schaefer_lss_df = pd.DataFrame.from_dict(cnr_schaefer_collector['lss'], orient='index', columns=schaefer_lut['regions'])

In [169]:
print("AVNR (schaefer, lsa) Mean: {}".format(cvnr_schaefer_lsa_df.mean().mean()))
print("AVNR (schaefer, lsa) Max: {}".format(cvnr_schaefer_lsa_df.mean().max()))

AVNR (schaefer, lsa) Mean: 1.5401114183170141
AVNR (schaefer, lsa) Max: 3.6112200244326864


In [170]:
print("CNR (schaefer, lsa) Mean: {}".format(cnr_schaefer_lsa_df.mean().mean()))
print("CNR (schaefer, lsa) Max: {}".format(cnr_schaefer_lsa_df.mean().max()))

CNR (schaefer, lsa) Mean: 1.020134793837807
CNR (schaefer, lsa) Max: 2.480575385177624


In [171]:
print("AVNR (schaefer, lss) Mean: {}".format(cvnr_schaefer_lss_df.mean().mean()))
print("AVNR (schaefer, lss) Max: {}".format(cvnr_schaefer_lss_df.mean().max()))

AVNR (schaefer, lss) Mean: 1.033641100411185
AVNR (schaefer, lss) Max: 2.327435377186426


In [172]:
print("CNR (schaefer, lss) Mean: {}".format(cnr_schaefer_lss_df.mean().mean()))
print("CNR (schaefer, lss) Max: {}".format(cnr_schaefer_lss_df.mean().max()))

CNR (schaefer, lss) Mean: 0.7137942709185943
CNR (schaefer, lss) Max: 1.8056619546708372


## PLAYGROUND

In [ ]:
np.median(nib.load(lss_taskswitch_single_objs[0].path).get_fdata()[response_mask].std(axis=1))

In [6]:
def _get_bold_file(layout, participant):
    return layout.get(
            subject=participant,
            suffix='bold',
            extension='nii.gz',
            desc='preproc',
            space='MNI152NLin2009cAsym',
            task='taskswitch',
            return_type='file')[0]

def _get_event_file(layout, participant):
    return layout.get(subject=participant,
                            suffix='events',
                            extension='tsv',
                            task='taskswitch',
                            return_type='file')[0]

def _get_confounds_file(layout, participant):
      return layout.get(subject=participant,
                            suffix='regressors',
                            extension='tsv',
                            desc='confounds',
                            task='taskswitch',
                            return_type='file')[0]

def _get_mask_file(layout, participant):
    return layout.get(subject=participant,
                      suffix='mask',
                      extension='nii.gz',
                        desc='brain',
                      space='MNI152NLin2009cAsym',
                        task='taskswitch',
                        return_type='file')[0]

In [7]:
lsa_beta_series = LSABetaSeries(
    bold_file=_get_bold_file(layout, "GE120001"),
    bold_metadata={"RepetitionTime": 2},
    confounds_file=_get_confounds_file(layout, "GE120001"),
    events_file=_get_event_file(layout, "GE120001"),
    high_pass=0.008,
    hrf_model="glover",
    mask_file=_get_mask_file(layout, "GE120001"),
    selected_confounds=mah_confounds,
    smoothing_kernel=None,
)

In [ ]:
lsa_beta_series.inputs

In [ ]:
res = lsa_beta_series.run()

In [ ]:
lsa_resid = nib.load(res.outputs.residual)

In [ ]:
np.median(lsa_resid.get_fdata()[response_mask].std(axis=1))

In [ ]:
res.outputs.beta_maps

In [ ]:
cnrs = nib.load(res.outputs.beta_maps[0]).get_fdata()[response_mask] / np.atleast_2d(lsa_resid.get_fdata()[response_mask].std(axis=1)).T

In [ ]:
np.nanmax(np.abs(cnrs))

In [ ]:
nib.load(lss_taskswitch_single_objs[0].path).get_fdata().mean(axis=-1).shape

In [ ]:
nib.load('/home/jdkent/hpchome/bids/derivatives/fmriprep/sub-GE120001/ses-pre/func/sub-GE120001_ses-pre_task-taskswitch_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz').shape

In [ ]:
def _calc_cnrs(participant, bold_file, mask_file, events_file, confounds_file, selected_confounds):
    lsa_beta_series = LSABetaSeries(
        bold_file=bold_file,
        bold_metadata={"RepetitionTime": 2},
        confounds_file=confounds_file,
        events_file=events_file,
        high_pass=0.008,
        hrf_model="glover",
        mask_file=mask_file,
        selected_confounds=selected_confounds,
        smoothing_kernel=None,
    )
    
    res = lsa_beta_series.run()
    result_collector = {'participant': participant}
    
    resid_img = nib.load(res.outputs.residual)
    noise_std = np.std(resid_img.get_fdata(), axis=-1)
    for beta_file in res.outputs.beta_maps:
        beta_data = nib.load(beta_file).get_fdata()
        trial_type = re.search(r'desc-([A-Za-z0-9]+)_', beta_file).groups()[0]
        # absolute value across trials
        amplitude = np.median(np.abs(beta_data), axis=-1)
        beta_std = np.std(beta_data, axis=-1)
        # voxelwise map of cnr and bnr
        cnr = amplitude / noise_std
        bnr = beta_std / noise_std
        result_collector[trial_type] = {'cnr': cnr, 'bnr': bnr}

    return result_collector
        

In [ ]:
init_sub = "GE120001"
p = init_sub
iter_tup = (p, _get_bold_file(layout, p), _get_mask_file(layout, p), _get_event_file(layout, p), _get_confounds_file(layout, p), mah_confounds)

In [ ]:
iter_list = [
  (f'{sub}',
   f'/home/jdkent/hpchome/bids/derivatives/fmriprep/sub-{sub}/ses-pre/func/sub-{sub}_ses-pre_task-taskswitch_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz',
  f'/home/jdkent/hpchome/bids/derivatives/fmriprep/sub-{sub}/ses-pre/func/sub-{sub}_ses-pre_task-taskswitch_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz',
  f'/home/jdkent/hpchome/bids/sub-{sub}/ses-pre/func/sub-{sub}_ses-pre_task-taskswitch_events.tsv',
  f'/home/jdkent/hpchome/bids/derivatives/fmriprep/sub-{sub}/ses-pre/func/sub-{sub}_ses-pre_task-taskswitch_desc-confounds_regressors.tsv',
  mah_confounds) for sub in layout.get_subjects()]

In [ ]:
iter_list[2]

In [ ]:
cnr_dict = [_calc_cnrs(*args) for args in iter_list]

In [ ]:
cnr_maxes = {}
bnr_maxes = {}
for cnr_d in cnr_dict:
    part = cnr_d["participant"]
    cnr_maxes[part] = {}
    bnr_maxes[part] = {}
    for trial_type in ["single", "repeat", "switch"]:
        cnr_maxes[part][trial_type] = np.nanmax(cnr_d[trial_type]["cnr"])
        bnr_maxes[part][trial_type] = np.nanmax(cnr_d[trial_type]["bnr"])
        

In [ ]:
cnr_max_arr = np.array([list(n.values()) for n in cnr_maxes.values()]).flatten()

In [ ]:
np.median(cnr_max_arr)

In [ ]:
bnr_max_arr = np.array([list(n.values()) for n in bnr_maxes.values()]).flatten()

In [ ]:
np.median(bnr_max_arr[bnr_max_arr < 100])

In [ ]:
cnr_medians = {}
bnr_medians = {}
for cnr_d in cnr_dict:
    part = cnr_d["participant"]
    cnr_medians[part] = {}
    bnr_medians[part] = {}
    for trial_type in ["single", "repeat", "switch"]:
        cnr_medians[part][trial_type] = np.nanmedian(cnr_d[trial_type]["cnr"][np.nonzero(cnr_d[trial_type]["cnr"])])
        bnr_medians[part][trial_type] = np.nanmedian(cnr_d[trial_type]["bnr"][np.nonzero(cnr_d[trial_type]["bnr"])])

In [ ]:
cnr_medians

In [ ]:
bnr_medians